<a href="https://colab.research.google.com/github/9yubean2/ArtificialIntelligence/blob/main/GAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [3]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.7.0


In [4]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3 MB 26 kB/s 
     |████████████████████████████████| 50 kB 5.5 MB/s 
     |████████████████████████████████| 3.8 MB 31.2 MB/s 
     |████████████████████████████████| 503 kB 45.7 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=6e7dfab45267fef5dfcfe6a484501803020b8eb7b1362df8b0eddbc2895e7b4c
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot=True)

# 옵션 설정

total_epoch = 100
batch_size = 100
learning_rate = 0.00001
n_hidden1 = 256
n_hidden2 = 128
n_input = 28 * 28
n_noise = 128

# 신경망 모델 구성

X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden1], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden1]))
G_W2 = tf.Variable(tf.random_normal([n_hidden1, n_hidden2], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_hidden2]))
G_W3 = tf.Variable(tf.random_normal([n_hidden2, n_hidden1], stddev=0.01))
G_b3 = tf.Variable(tf.zeros([n_hidden1]))
G_W4 = tf.Variable(tf.random_normal([n_hidden1, n_input], stddev=0.01))
G_b4 = tf.Variable(tf.zeros([n_input]))


D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden1], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden1]))
D_W2 = tf.Variable(tf.random_normal([n_hidden1, n_hidden2], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([n_hidden2]))
D_W3 = tf.Variable(tf.random_normal([n_hidden2, 1], stddev=0.01))
D_b3 = tf.Variable(tf.zeros([1]))

def generator(noise_z):
  hidden_1 = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
  hidden_2 = tf.nn.relu(tf.matmul(hidden_1, G_W2) + G_b2)
  hidden_3 = tf.nn.relu(tf.matmul(hidden_2, G_W3) + G_b3)
  output = tf.nn.sigmoid(tf.matmul(hidden_3, G_W4) + G_b4)

  return output

def discriminator(inputs):
  hidden_1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
  hidden_2 = tf.nn.relu(tf.matmul(hidden_1, D_W2) + D_b2)
  output = tf.nn.sigmoid(tf.matmul(hidden_2, D_W3) + D_b3)

  return output

def get_noise(batch_size, n_noise):
  return np.random.normal(size=(batch_size, n_noise))

G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene))

D_var_list = [D_W1, D_b1, D_W2, D_b2, D_W3, D_b3]
G_var_list = [G_W1, G_b1, G_W2, G_b2, G_W3, G_b3, G_W4, G_b4]

train_D = tf.train.RMSPropOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.RMSPropOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

# 신경망 모델 학습

sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    noise = get_noise(batch_size, n_noise)
    
    _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
    _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
  
  print('Epoch:', '%04d' % epoch, 'D loss: {:.4}'.format(loss_val_D), 
        'G loss: {:.4}'.format(loss_val_G))

  # 확인용 이미지 생성

  if epoch == 0 or (epoch + 1) % 10 == 0:
    sample_size = 10
    noise = get_noise(sample_size, n_noise)
    samples = sess.run(G, feed_dict={Z: noise})

    fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

    for i in range(sample_size):
      ax[i].set_axis_off()
      ax[i].imshow(np.reshape(samples[i], (28, 28)))

    plt.savefig('./sample/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
    # samples 폴더 생성

    plt.close(fig)

print('최적화 완료!')

ModuleNotFoundError: ignored